<img src='../../img/anaconda-logo.png' align='left' style="padding:10px">
<br>
*Copyright Continuum 2012-2016 All Rights Reserved.*

# Bokeh Plotting and Custom Tools

## Table of Contents
* [Bokeh Plotting and Custom Tools](#Bokeh-Plotting-and-Custom-Tools)
	* [Set-Up](#Set-Up)
* [ColumnDataSource](#ColumnDataSource)
	* [Bokeh Examples](#Bokeh-Examples)
* [ColumnDataSource Demonstration: Election Results](#ColumnDataSource-Demonstration:-Election-Results)
	* [Map Boundaries](#Map-Boundaries)
	* [Load Data to Plot on Map](#Load-Data-to-Plot-on-Map)
	* [Use Data to create ColumnDataSource](#Use-Data-to-create-ColumnDataSource)
	* [Add Custom Interactive Tools](#Add-Custom-Interactive-Tools)
		* [HoverTool](#HoverTool)
* [Summary](#Summary)
* [Exercise](#Exercise)


## Set-Up

In [ ]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
output_notebook()

# ColumnDataSource

Let's introduce `ColumnDataSource` as a preface before diving into this example. 

* `ColumnDataSource` is **THE** main object needed to understand rows and columns in the Bokeh world.
* `ColumnDataSource` is a wrapper around a Pandas DataFrame
* In order to integrate deeper with Bokeh, we need to make a `ColumnDataSource` object. 

## Bokeh Examples

Bokeh in github has over 100 examples:

* https://github.com/bokeh/bokeh/tree/master/examples
* https://github.com/bokeh/bokeh/blob/master/examples/plotting/file/airports_map.py

# ColumnDataSource Demonstration: Election Results

<div class='alert alert-info'>
<img src='img/topics/Best-Practice.png' align='left' style='padding:10px'>
<br><big><big>
Load *all* of the plot information in a Pandas DataFrame.
</big></big>
<br><br>
</div>

To visualize election results for every county in Pennsylvania we need
* county boundaries
* per-county color for winning party

Further, we can make use of
* per-county percentage of votest to each candidate
* county name

## Map Boundaries

First, get the latitude and longitude **borders** for every county in PA

`bokeh.sampledata.us_counties` contains location and boundary data for all US counties!

In [ ]:
from bokeh.sampledata.us_counties import data

In [ ]:
us_counties = pd.DataFrame(data).T
pennsylvania=us_counties.loc[us_counties['state']=='pa']
pennsylvania.head()

## Load Data to Plot on Map

Election results from all US counties

In [ ]:
election_2012 = pd.read_csv('data/2012Election.csv')
pa_results = election_2012.loc[election_2012['state']=='PA',['Obama','Romney','county']]
pa_results.head()

In [ ]:
pa2012 = pd.merge(pennsylvania, pa_results, left_on='name', right_on='county').drop(['name','state'], axis='columns')
pa2012['Other']=100-(pa2012['Romney']+pa2012['Obama'])
pa2012.head(1)

Now I create a color column that I will use to color each county in my plot.

In [ ]:
def blue_or_red(x):
    if x['Romney'] > x['Obama']:
        return 'red'
    elif x['Obama'] > x['Romney']:
        return 'blue'
    else:
        return 'purple'

In [ ]:
pa2012['color'] = pa2012.apply(blue_or_red, axis=1)
pa2012.head()

## Use Data to create ColumnDataSource

In order to integrate deeper with Bokeh I need to make a ColumnDataSource object. There is a direct mapping from a Pandas DataFrame to a Bokeh ColumnDataSource.

In [ ]:
from bokeh.charts import ColumnDataSource
source = ColumnDataSource(pa2012)

The ColumnDataSource is passed to the plotting function with `source=` and I can now access the columns by strings.

In [ ]:
from bokeh.plotting import figure
plot = figure(width=850)
plot.patches('lons', 'lats', color='color', line_color='white', source=source)
show(plot)

## Add Custom Interactive Tools

Now that I have a `ColumnDataSource` object I can use it's information to prepare a custom hover tool that will show the county name and the percentage of votes cast for each candidate.

I'm going to import all of the tool classes I need to use.

In [ ]:
from bokeh.models import PanTool,WheelZoomTool,ResetTool,HoverTool

When creating a Bokeh `figure()`, the tools from `bokeh.models` can be passed as instances or as comma-delimited string. 

You can also set these after you create the `figure` object.

Here are some examples:

### HoverTool

* `tooltips` needs a list of tuples, where each tuple is (dispay_text,column_name)
* `source` is from `ColumnDataSource(pa2012)`
* Recall that the entire data set was 
  `election_2012 = pd.read_csv('data/2012Election.csv')`
* then we subset to just get the Pennsylvania data with `election_2012['state']=='PA'

In [ ]:
hover = HoverTool(
  tooltips = [
        #dispay_text,column_name
        ('County'   ,'@county'),
        ('% Obama'  ,'@Obama'),
        ('% Romeny' ,'@Romney'),
        ('% Other'  ,'@Other')
    ]
)

tools=[PanTool(), WheelZoomTool(), hover, ResetTool()]

plot = figure(tools=tools, width=850)
plot.patches('lons', 'lats', color='color', line_color='white', source=source)
show(plot)

How decisive were the voters?
-----------------------------

Create a function to be applied to the data:

In [ ]:
def set_alpha(x):
    if x['color'] == 'blue':
        return 1 - x['Romney'] / x['Obama']
    else:
        return 1 - x['Obama'] / x['Romney']

Apply the function so as to return red or blue depending on the data

*Note: `axis="columns"` is the same as `axis=1` and **BOTH** can be confusing! http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html*

In [ ]:
pa2012['alpha'] = pa2012.apply(set_alpha, axis='columns')
source = ColumnDataSource(pa2012)
pa2012.head()

Completely transparent counties are 50/50 split between Obama and Romney.

In [ ]:
hover = HoverTool(
  tooltips = [
        #dispay_text,column_name
        ('County'   ,'@county'),
        ('% Obama'  ,'@Obama'),
        ('% Romeny' ,'@Romney'),
        ('% Other'  ,'@Other')
    ]
)

tools=[PanTool(), WheelZoomTool(), hover, ResetTool()]

plot = figure(tools=tools, width=850)
plot.patches('lons', 'lats', color='color', line_color='white', alpha='alpha', source=source)
show(plot)

# Summary

Put all of the data in your DataFrame
* ColumnDataSource objects are easy to make

Look at the help for the tool objects. They have interesting parameters.

# Exercise

<img src='img/topics/Exercise.png' align='left' style='padding:10px'>
<br>
<a href='./Bokeh_ex_interactive.ipynb' class='btn btn-primary btn-lg'>Interactive tools</a>

----
<a href='./Bokeh_maps.ipynb' class='btn btn-primary'>Map overlays</a>

---
*Copyright Continuum 2012-2016 All Rights Reserved.*